In [22]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import yt
import sys
sys.path.append('./../')
#import flashy

%matplotlib inline
plt.style.use('./../gad_noltx.mplstyle')

In [20]:
EOSIMAX = 261 # This is the number of steps in density (I axis/rows)
EOSJMAX = 101 # This is the number of steps in temperature (J axis/columns)
logT_lo = 3.0 # Minimum temperature in the table in log
logT_hi = 13.0 # Maximum temperature in the table in log
logd_lo = -12.0 # Minimum density in the table in log
logd_hi = 14.0 # Maximum density in the table in log
T_step = (logT_hi - logT_lo)/float(EOSJMAX-1)
d_step = (logd_hi - logd_lo)/float(EOSIMAX-1)

eos_f = np.zeros((EOSIMAX, EOSJMAX))
eos_fd = np.zeros((EOSIMAX, EOSJMAX))
eos_ft = np.zeros((EOSIMAX, EOSJMAX))
eos_fdd = np.zeros((EOSIMAX, EOSJMAX))
eos_ftt = np.zeros((EOSIMAX, EOSJMAX))
eos_fdt = np.zeros((EOSIMAX, EOSJMAX))
eos_fddt = np.zeros((EOSIMAX, EOSJMAX))
eos_fdtt = np.zeros((EOSIMAX, EOSJMAX))
eos_fddtt = np.zeros((EOSIMAX, EOSJMAX))

eos_dpdf = np.zeros((EOSIMAX, EOSJMAX))
eos_dpdfd = np.zeros((EOSIMAX, EOSJMAX))
eos_dpdft = np.zeros((EOSIMAX, EOSJMAX))
eos_dpdfdt = np.zeros((EOSIMAX, EOSJMAX))

eos_ef = np.zeros((EOSIMAX, EOSJMAX))
eos_efd = np.zeros((EOSIMAX, EOSJMAX))
eos_eft = np.zeros((EOSIMAX, EOSJMAX))
eos_efdt = np.zeros((EOSIMAX, EOSJMAX))

eos_xf = np.zeros((EOSIMAX, EOSJMAX))
eos_xfd = np.zeros((EOSIMAX, EOSJMAX))
eos_xft = np.zeros((EOSIMAX, EOSJMAX))
eos_xfdt = np.zeros((EOSIMAX, EOSJMAX))

In [29]:
# Read helm_table.dat
# Fastest way to read the table according to ChatGPT
with open('helm_table.dat', 'r') as helm_table:
    # Read the free energy table
    arrays = [np.empty((EOSIMAX,EOSJMAX), dtype=np.float64) for _ in range(9)]
    for j in range(EOSJMAX):
        for i in range(EOSIMAX):
            values = np.fromstring(helm_table.readline(), dtype=np.float64, sep=' ')
            for k in range(9):
                arrays[k][i,j] = values[k]
    eos_f, eos_fd, eos_ft, eos_fdd, eos_ftt, eos_fdt, eos_fddt, eos_fdtt, eos_fddtt = arrays

    # Read the pressure derivative table
    arrays = [np.empty((EOSIMAX,EOSJMAX), dtype=np.float64) for _ in range(4)]
    for j in range(EOSJMAX):
        for i in range(EOSIMAX):
            values = np.fromstring(helm_table.readline(), dtype=np.float64, sep=' ')
            for k in range(4):
                arrays[k][i,j] = values[k]
    eos_dpdf, eos_dpdfd, eos_dpdft, eos_dpdfdt = arrays

    # Read the chemical potential table
    arrays = [np.empty((EOSIMAX,EOSJMAX), dtype=np.float64) for _ in range(4)]
    for j in range(EOSJMAX):
        for i in range(EOSIMAX):
            values = np.fromstring(helm_table.readline(), dtype=np.float64, sep=' ')
            for k in range(4):
                arrays[k][i,j] = values[k]
    eos_ef, eos_efd, eos_eft, eos_efdt = arrays

    # Read the number density table
    arrays = [np.empty((EOSIMAX,EOSJMAX), dtype=np.float64) for _ in range(4)]
    for j in range(EOSJMAX):
        for i in range(EOSIMAX):
            values = np.fromstring(helm_table.readline(), dtype=np.float64, sep=' ')
            for k in range(4):
                arrays[k][i,j] = values[k]
    eos_xf, eos_xfd, eos_xft, eos_xfdt = arrays

In [31]:
# psi0 and its derivatives
psi0   = lambda zFunc: zFunc**3 * (zFunc * (-6.0*zFunc + 15.0) - 10.0) + 1.0
dpsi0  = lambda zFunc: zFunc**2 * (zFunc * (-30.0*zFunc + 60.0) - 30.0)
ddpsi0 = lambda zFunc: zFunc * (zFunc * (-120.0*zFunc + 180.0) - 60.0)

# psi1 and its derivatives
psi1   = lambda zFunc: zFunc * (zFunc**2 * (zFunc * (-3.0*zFunc + 8.0) - 6.0) + 1.0)
dpsi1  = lambda zFunc: zFunc**2 * (zFunc * (-15.0*zFunc + 32.0) - 18.0) + 1.0
ddpsi1 = lambda zFunc: zFunc * (zFunc * (-60.0*zFunc + 96.0) - 36.0)

# psi2 and its derivatives
psi2   = lambda zFunc: 0.5 * zFunc**2 * (zFunc * (zFunc * (-zFunc + 3.0) - 3.0) + 1.0)
dpsi2  = lambda zFunc: 0.5 * zFunc * (zFunc * (zFunc * (-5.0*zFunc + 12.0) - 9.0) + 2.0)
ddpsi2 = lambda zFunc: 0.5 * (zFunc * (zFunc * (-20.0*zFunc + 36.0) - 18.0) + 2.0)


def herm5(w0t, w1t, w2t, w0mt, w1mt, w2mt, w0d, w1d, w2d, w0md, w1md, w2md, fi):
    w_factors = np.array([
        w0d * w0t,   w0md * w0t,   w0d * w0mt,   w0md * w0mt,
        w0d * w1t,   w0md * w1t,   w0d * w1mt,   w0md * w1mt,
        w0d * w2t,   w0md * w2t,   w0d * w2mt,   w0md * w2mt,
        w1d * w0t,   w1md * w0t,   w1d * w0mt,   w1md * w0mt,
        w2d * w0t,   w2md * w0t,   w2d * w0mt,   w2md * w0mt,
        w1d * w1t,   w1md * w1t,   w1d * w1mt,   w1md * w1mt,
        w2d * w1t,   w2md * w1t,   w2d * w1mt,   w2md * w1mt,
        w1d * w2t,   w1md * w2t,   w1d * w2mt,   w1md * w2mt,
        w2d * w2t,   w2md * w2t,   w2d * w2mt,   w2md * w2mt
    ])

    return np.dot(fi, w_factors)


# Bicubic hermite polynomial for the electron pressure derivatives
def herm3dpd(i, j, w0t, w1t, w0mt, w1mt, w0d, w1d, w0md, w1md):
    eos_values = np.array([
        eos_dpdf[i, j],     eos_dpdf[i+1, j],     eos_dpdf[i, j+1],     eos_dpdf[i+1, j+1],
        eos_dpdft[i, j],    eos_dpdft[i+1, j],    eos_dpdft[i, j+1],    eos_dpdft[i+1, j+1],
        eos_dpdfd[i, j],    eos_dpdfd[i+1, j],    eos_dpdfd[i, j+1],    eos_dpdfd[i+1, j+1],
        eos_dpdfdt[i, j],   eos_dpdfdt[i+1, j],   eos_dpdfdt[i, j+1],   eos_dpdfdt[i+1, j+1]
    ])

    weight_factors = np.array([
        w0d * w0t,   w0md * w0t,   w0d * w0mt,   w0md * w0mt,
        w0d * w1t,   w0md * w1t,   w0d * w1mt,   w0md * w1mt,
        w1d * w0t,   w1md * w0t,   w1d * w0mt,   w1md * w0mt,
        w1d * w1t,   w1md * w1t,   w1d * w1mt,   w1md * w1mt
    ])

    return np.dot(eos_values, weight_factors)


# Bicubic hermite polynomial for the electron chemical potential
def herm3e(i, j, w0t, w1t, w0mt, w1mt, w0d, w1d, w0md, w1md):
    eos_values = np.array([
        eos_ef[i, j],   eos_ef[i+1, j],   eos_ef[i, j+1],   eos_ef[i+1, j+1],
        eos_eft[i, j],  eos_eft[i+1, j],  eos_eft[i, j+1],  eos_eft[i+1, j+1],
        eos_efd[i, j],  eos_efd[i+1, j],  eos_efd[i, j+1],  eos_efd[i+1, j+1],
        eos_efdt[i, j], eos_efdt[i+1, j], eos_efdt[i, j+1], eos_efdt[i+1, j+1]
    ])

    weight_factors = np.array([
        w0d * w0t,   w0md * w0t,   w0d * w0mt,   w0md * w0mt,
        w0d * w1t,   w0md * w1t,   w0d * w1mt,   w0md * w1mt,
        w1d * w0t,   w1md * w0t,   w1d * w0mt,   w1md * w0mt,
        w1d * w1t,   w1md * w1t,   w1d * w1mt,   w1md * w1mt
    ])

    return np.dot(eos_values, weight_factors)

# Bicubic hermite polynomial for the electron positron number densities
def herm3x(i, j, w0t, w1t, w0mt, w1mt, w0d, w1d, w0md, w1md):
    eos_values = np.array([
        eos_xf[i, j],   eos_xf[i+1, j],   eos_xf[i, j+1],   eos_xf[i+1, j+1],
        eos_xft[i, j],  eos_xft[i+1, j],  eos_xft[i, j+1],  eos_xft[i+1, j+1],
        eos_xfd[i, j],  eos_xfd[i+1, j],  eos_xfd[i, j+1],  eos_xfd[i+1, j+1],
        eos_xfdt[i, j], eos_xfdt[i+1, j], eos_xfdt[i, j+1], eos_xfdt[i+1, j+1]
    ])

    weight_factors = np.array([
        w0d * w0t,   w0md * w0t,   w0d * w0mt,   w0md * w0mt,
        w0d * w1t,   w0md * w1t,   w0d * w1mt,   w0md * w1mt,
        w1d * w0t,   w1md * w0t,   w1d * w0mt,   w1md * w0mt,
        w1d * w1t,   w1md * w1t,   w1d * w1mt,   w1md * w1mt
    ])

    return np.dot(eos_values, weight_factors)

In [27]:
dict = {'name':[0,1,2], 'test':[3,4,5]}
dtype = [(key, int) for key in dict.keys()]
print(np.array(list(zip(*dict.values())), dtype=dtype)['name'])
dtype = ['r']
dtype += list(dict.keys())
print(dtype)

[0 1 2]
['r', 'name', 'test']
